In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


2. 定义键盘布局的特征表示
为了将键盘布局输入到神经网络中，需要将其转换为数值特征。可以采用以下方法：

One-Hot 编码：将每个键位上的字符用 one-hot 向量表示，整个键盘展开为一个长向量。

In [ ]:
def layout_to_features(layout_str):
    # 定义字符集（包含所有可能的字符）
    charset = sorted(list(set(''.join(layout.values()))))
    char_to_index = {char: idx for idx, char in enumerate(charset)}
    feature = []
    for char in layout_str:
        one_hot = [0] * len(charset)
        index = char_to_index[char]
        one_hot[index] = 1
        feature.extend(one_hot)
    return feature


3. 定义深度学习模型

In [ ]:
class FitnessPredictor(nn.Module):
    def __init__(self, input_size):
        super(FitnessPredictor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # 输出预测的适应度值
        )
    
    def forward(self, x):
        return self.fc(x)


4. 初始化模型和优化器

In [ ]:
# 计算输入特征的维度
input_size = len(layout_to_features(layout['qwerty']))

model = FitnessPredictor(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


5. 修改 get_evals 函数
在评估适应度时，使用深度学习模型进行预测：

In [ ]:
def get_evals(population, use_model=False):
    evals = {}
    features = []
    indices = []
    for i, genome in enumerate(population):
        genome_str = ''.join(genome)
        if use_model:
            # 使用模型预测适应度
            feature = layout_to_features(genome_str)
            features.append(feature)
            indices.append(i)
        else:
            # 真实计算适应度
            keyboard = genome_to_keyboard(genome)
            dist = total_distance_with_penalty(total_full_text, keyboard, penalty=1)
            repeat_count = finger_repeat(total_full_text, keyboard)
            pop_hamming_dist = cal_layout_hamming_dist(layout['qwerty'], genome)
            pop_punish_dist = cal_layout_punish_distance(layout['qwerty'], genome)
            evals[i] = int((dist + repeat_count * 2) * (pop_hamming_dist + pop_punish_dist * 0.5 + 10) / 200000)
    
    if use_model and features:
        # 转换为 Tensor
        features_tensor = torch.tensor(features, dtype=torch.float32)
        # 模型预测
        with torch.no_grad():
            predictions = model(features_tensor).squeeze().numpy()
        for idx, pred in zip(indices, predictions):
            evals[idx] = pred.item()
    
    # 对 evals 进行排序
    sorted_evals = [k for k, v in sorted(evals.items(), key=lambda item: item[1])]
    return evals, sorted_evals
